In [1]:
import os

In [2]:
%pwd

'd:\\PROJECTS\\Text_summariser\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\PROJECTS\\Text_summariser'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelUploaderConfig:
    model_path: Path
    tokenizer_path: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_uploader_config(self) -> ModelUploaderConfig:
        config = self.config.model_uploader
        # params = self.params.TrainingArguments

        # create_directories([config.root_dir])

        model_uploader_config = ModelUploaderConfig(
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path
            
        )

        return model_uploader_config

In [9]:
from huggingface_hub import HfApi, HfFolder
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

c:\Users\MARK\anaconda3\envs\textVenv\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\MARK\anaconda3\envs\textVenv\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [10]:
import torch

In [11]:
class ModelUploader:
    def __init__(self, config: ModelUploaderConfig):
        self.config = config
        
    def upload_model(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        api = HfApi()

        token = HfFolder.get_token()

        repo_name = 'maichmarc/textS'

        api.create_repo(repo_id=repo_name, token=token)

        model_pegasus.push_to_hub(repo_name)
        tokenizer.push_to_hub(repo_name)

        

In [14]:
try:
    config = ConfigurationManager()
    model_uploader_config = config.get_model_uploader_config()
    model_uploader = ModelUploader(config=model_uploader_config)
    model_uploader.upload_model()
except Exception as e:
    raise e

model.safetensors:   2%|▏         | 50.3M/2.28G [00:25<18:28, 2.02MB/s]   

KeyboardInterrupt: 

model.safetensors:   2%|▏         | 50.6M/2.28G [00:40<18:28, 2.02MB/s]